In [1]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [17]:
print(gym.envs.registry.keys())

dict_keys(['CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v1', 'Acrobot-v1', 'LunarLander-v2', 'LunarLanderContinuous-v2', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v2', 'Blackjack-v1', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'CliffWalking-v0', 'Taxi-v3', 'Reacher-v2', 'Reacher-v4', 'Pusher-v2', 'Pusher-v4', 'InvertedPendulum-v2', 'InvertedPendulum-v4', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulum-v4', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetah-v4', 'Hopper-v2', 'Hopper-v3', 'Hopper-v4', 'Swimmer-v2', 'Swimmer-v3', 'Swimmer-v4', 'Walker2d-v2', 'Walker2d-v3', 'Walker2d-v4', 'Ant-v2', 'Ant-v3', 'Ant-v4', 'Humanoid-v2', 'Humanoid-v3', 'Humanoid-v4', 'HumanoidStandup-v2', 'HumanoidStandup-v4'])


In [3]:
pip install --upgrade gym==0.19.0

     |████████████████████████████████| 1.6 MB 2.2 MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.19.0-py3-none-any.whl size=1663119 sha256=6e6da54ae857f2d99f5cf3c22b1fa2d5c47897c741d40e424b687114d9aa3b60
  Stored in directory: /Users/roy/Library/Caches/pip/wheels/11/36/28/628f4dd3779e4037a6fca1aaed76827ffa4315c3ab6bfadcf6
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.26.2
    Uninstalling gym-0.26.2:
      Successfully uninstalled gym-0.26.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ale-py

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ale_py import ALEInterface

ale = ALEInterface()

In [8]:
pip install atari_py

     |████████████████████████████████| 2.4 MB 163 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install "gym[atari]"


  Using cached atari-py-0.2.6.tar.gz (790 kB)
Reason for being yanked: re-release with new wheels
  ERROR: Command errored out with exit status 1:
   command: /Users/roy/opt/anaconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/_4/473sklf50p9_zw8_mv1wqcqm0000gn/T/pip-install-524878g1/atari-py_7c8581deb2f94b4498826ef80b351df3/setup.py'"'"'; __file__='"'"'/private/var/folders/_4/473sklf50p9_zw8_mv1wqcqm0000gn/T/pip-install-524878g1/atari-py_7c8581deb2f94b4498826ef80b351df3/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/_4/473sklf50p9_zw8_mv1wqcqm0000gn/T/pip-wheel-ul4g0bcs
       cwd: /private/var/folders/_4/473sklf50p9_zw8_mv1wqcqm0000gn/T/pip-install-524878g1/atari-py_7c8581deb2f94b4498826ef80b351df3/
  Complete output (502 lines):
  running bdist_wheel
  running build
  runnin

In [5]:

env = gym.make("Assault-v0")

In [6]:
import gym
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from collections import namedtuple, deque
import time

In [7]:
class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)
        self.transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'done'))

    def push(self, *args):
        """Save a transition"""
        self.memory.append(self.transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


class DQN(keras.Model):

    def __init__(self, n_actions):
        super(DQN, self).__init__()

        self.layer1 = layers.Conv2D(16, 5, strides=2, activation="relu")
        self.bn1 = layers.BatchNormalization()
        self.layer2 = layers.Conv2D(16, 5, strides=2, activation="relu")
        self.bn2 = layers.BatchNormalization()
        self.layer3 = layers.Conv2D(32, 5, strides=2, activation="relu")
        self.bn3 = layers.BatchNormalization()
        self.flatten = layers.Flatten()
        self.layer4 = layers.Dense(512, activation="relu")
        self.action = layers.Dense(n_actions, activation="linear")

    def call(self, inputs):
        x = self.layer1(inputs)
        x = self.bn1(x)
        x = self.layer2(x)
        x = self.bn2(x)
        x = self.layer3(x)
        x = self.bn3(x)
        x = self.flatten(x)
        x = self.layer4(x)
        return self.action(x)

In [8]:
BATCH_SIZE = 128
GAMMA = 0.999
EPSILON = 0.9
EPSILON_END = 0.05
EPSILON_DECAY = 2000
TARGET_UPDATE = 5
EPISODES = 30
n_actions = env.action_space.n



In [9]:
model = DQN(n_actions)
model_target = DQN(n_actions)
memory = ReplayMemory(10000)

optimizer = keras.optimizers.Adam(lr=2.5e-4, clipnorm=1.0)
loss_function = keras.losses.Huber()




In [14]:
def take_action(state, EPSILON):
    sample = random.random()
    if sample > (1-EPSILON):
        action = env.action_space.sample()
    else:
        action = np.argmax(model(state))
    next_state, reward, done, _ = env.step(action)
    return next_state/255., reward, done, action


def optimize_model():
    if memory.__len__() < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = memory.transition(*zip(*transitions))

    state_batch = np.array(batch.state)
    action_batch = np.array(batch.action)
    next_state_batch = np.array(batch.next_state)
    rewad_batch = np.array(batch.reward)
    done_batch = np.array(batch.done, dtype=np.int8)

    future_rewards = model_target(next_state_batch)
    target = rewad_batch + GAMMA * tf.reduce_max(future_rewards, axis=-1) * (1 - done_batch)

    action_mask = tf.one_hot(action_batch, n_actions)

    with tf.GradientTape() as tape:
        q_values = model(state_batch)
        q_action = tf.reduce_sum(tf.multiply(q_values, action_mask), axis=-1)
        loss = loss_function(target, q_action)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    #model.save_weights('/Users/roy/Desktop/UNI')




In [16]:
for episode in range(1, EPISODES + 1):
    start = time.time()
    state = env.reset() / 255.
    done = False
    episode_reward = 0

    while not done:
        # env.render()
        print(".", end="")
        next_state, reward, done, action = take_action(np.expand_dims(state, 0), EPSILON)
        print (reward)
        memory.push(state, action, next_state, reward, done)
        optimize_model()
        EPSILON -= EPSILON / EPSILON_DECAY
        EPSILON = max(EPSILON, EPSILON_END)
        episode_reward += reward
        state = next_state

    print(f"\nEpisodio: {episode} - Recompensa: {episode_reward} - {(time.time()-start)/60:2.2f} min")

    if episode % TARGET_UPDATE == 0:
        print("TARGET MODEL UPDATED...")
        model_target.set_weights(model.get_weights())

env.close()

.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.21.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.21.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.0
.0.

KeyboardInterrupt: 